In [1]:
import sys
sys.path
sys.path.append('/home/helge/code/back_of_the_envelope')

import pandas as pd
import random

df = pd.read_html('https://www.wahlrecht.de/umfragen/landtage/bremen.htm')[1]
allensbach = pd.read_html('https://www.wahlrecht.de/umfragen/allensbach.htm')[1]

parties = ['CDU', 'SPD', 'GRÜNE', 'FDP', 'LINKE', 'AfD', 'Sonstige']
row = df.iloc[0]
polling_data = {
    p: int(row[p].split(" ")[0])
for p in parties
}

In [2]:
from forecasters.election import ElectionForecaster

bremen_forecaster = ElectionForecaster(polling_data)
next(bremen_forecaster.predict(1))

{'CDU': 22.197309417040348,
 'SPD': 37.89237668161434,
 'GRÜNE': 22.42152466367712,
 'FDP': 8.968609865470869,
 'LINKE': 8.520179372197326}

In [3]:
in_parliament = {p: 0 for p in parties}

number_red_green = 0

for x in bremen_forecaster.predict(1000):
    for p in x.keys():
        in_parliament[p] += 1
    for p, v in x.items():
        if v > 50:
            print(f"{p} has majority")

    if x["SPD"] + x["GRÜNE"] > 50:
        number_red_green += 1

print(in_parliament)
print(number_red_green)

{'CDU': 1000, 'SPD': 1000, 'GRÜNE': 1000, 'FDP': 871, 'LINKE': 996, 'AfD': 836, 'Sonstige': 0}
976


In [11]:
df = pd.read_html('https://www.wahlrecht.de/umfragen/landtage/bayern.htm')[1]
row = df.iloc[0]
parties = ['CSU',
       'SPD', 'GRÜNE', 'FDP', 'LINKE', 'FW', 'AfD', 'Sonstige']
polling_data = {
    p: int(row[p].split(" ")[0])
       for p in parties
}
bayern_forecaster = ElectionForecaster(polling_data)

in_parliament = {p: 0 for p in parties}

number_red_green = 0

for x in bayern_forecaster.predict(1000):
    for p in x.keys():
        in_parliament[p] += 1
    for p, v in x.items():
        if v > 50:
            print(f"{p} has majority")

    if x["SPD"] + x["GRÜNE"] > 50:
        number_red_green += 1

print(in_parliament)
print(number_red_green)

{'CSU': 1000, 'SPD': 1000, 'GRÜNE': 1000, 'FDP': 853, 'LINKE': 0, 'FW': 1000, 'AfD': 999, 'Sonstige': 0}
0


In [14]:
from itertools import combinations

in_parliament = {p: 0 for p in parties}
majorities = {}

number_red_green = 0

for x in bayern_forecaster.predict(1000):
    for p in x.keys():
        in_parliament[p] += 1
    for p, v in x.items():
        if v > 50:
            print(f"{p} has majority")

    pip = list(x.keys())

    for a, b in combinations(pip, 2):
        if x[a] + x[b] > 50:
            if f"{a}_{b}" in majorities:
                majorities[f"{a}_{b}"] += 1
            else:
                majorities[f"{a}_{b}"] = 1

print(in_parliament)
print(majorities)

{'CSU': 1000, 'SPD': 1000, 'GRÜNE': 1000, 'FDP': 850, 'LINKE': 0, 'FW': 1000, 'AfD': 1000, 'Sonstige': 0}
{'CSU_SPD': 844, 'CSU_GRÜNE': 1000, 'CSU_FW': 712, 'CSU_AfD': 832, 'CSU_FDP': 200}
